<a href="https://colab.research.google.com/github/disha-cpu/Resources-For-Stat-Data/blob/master/3D_DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import plotly.express as px
import random
import math
%matplotlib inline

In [ ]:
data = pd.read_csv('/content/weatherGOA.csv')
data = data.replace(to_replace ="24:00:00", value ="00:00")
data['time'] = data['Date'] +" "+ data['UT time'] 
data['time'] = pd.to_datetime(data['time'])
data = data[data['time'].dt.minute == 0]
data['Temperature (C)'] = data['Temperature (K)'] - 273.15
data['Hour'] = data['time'].dt.hour.astype(float)
dummy = data[['Hour',"Temperature (C)","Relative Humidity (%)"]]
dummy

,Hour,Temperature (C),Relative Humidity (%)
0,0.0,26.35,88.12
4,1.0,26.39,87.56
8,2.0,26.66,86.70
12,3.0,27.00,85.48
16,4.0,27.32,84.08
...,...,...,...
108076,19.0,26.38,87.77
108080,20.0,26.30,87.67
108084,21.0,26.25,87.40
108088,22.0,26.24,87.11


In [ ]:
import statistics
statistics.mean(dummy['Temperature (C)'])

27.13796625222027

In [ ]:
statistics.mean(dummy['Relative Humidity (%)'])

70.92891614860865

In [ ]:
import collections
collections.Counter(round(dummy['Temperature (C)']))

Counter({19.0: 7,
         20.0: 52,
         21.0: 209,
         22.0: 528,
         23.0: 983,
         24.0: 1582,
         25.0: 3284,
         26.0: 4971,
         27.0: 5058,
         28.0: 3701,
         29.0: 1947,
         30.0: 1542,
         31.0: 1232,
         32.0: 1011,
         33.0: 559,
         34.0: 278,
         35.0: 70,
         36.0: 10})

In [ ]:
collections.Counter(round(dummy['Relative Humidity (%)']))

Counter({19.0: 1,
         20.0: 1,
         21.0: 4,
         22.0: 4,
         23.0: 9,
         24.0: 18,
         25.0: 27,
         26.0: 30,
         27.0: 24,
         28.0: 50,
         29.0: 51,
         30.0: 66,
         31.0: 65,
         32.0: 94,
         33.0: 89,
         34.0: 128,
         35.0: 142,
         36.0: 154,
         37.0: 161,
         38.0: 166,
         39.0: 176,
         40.0: 192,
         41.0: 220,
         42.0: 212,
         43.0: 190,
         44.0: 203,
         45.0: 221,
         46.0: 230,
         47.0: 222,
         48.0: 256,
         49.0: 242,
         50.0: 284,
         51.0: 256,
         52.0: 273,
         53.0: 314,
         54.0: 322,
         55.0: 320,
         56.0: 332,
         57.0: 342,
         58.0: 337,
         59.0: 347,
         60.0: 377,
         61.0: 372,
         62.0: 352,
         63.0: 353,
         64.0: 392,
         65.0: 386,
         66.0: 398,
         67.0: 418,
         68.0: 401,
         69.0: 431,


In [ ]:
# dummy = dataset.copy()
px.scatter_3d(dummy, x='Hour', y='Temperature (C)', z='Relative Humidity (%)').update_traces(marker={'size': 2})


In [ ]:
# d1 = dummy.copy()
dummy[['Hour',"Temperature (C)","Relative Humidity (%)"]] = StandardScaler().fit_transform(dummy.values)

In [ ]:
mins = len(dummy)//1000
nn = NearestNeighbors(n_neighbors=mins+1)

nn.fit(dummy[['Hour',"Temperature (C)","Relative Humidity (%)"]])

distances, neighbors = nn.kneighbors(dummy[['Hour',"Temperature (C)","Relative Humidity (%)"]])

# sort the distances
distances = np.sort(distances[:, mins], axis = 0)

#plot the distances
distances_df = pd.DataFrame({"distances": distances,
                             "index": list(range(0,len(distances)))})

def calc_distance(x1, y1, a, b, c):
  d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
  return d

def find_eps():
  # (y1 – y2)x + (x2 – x1)y + (x1y2 – x2y1) = 0
  a = distances[0] - distances[-1]  #y
  b = distances_df.index[-1] - distances_df.index[0]    #x
  c1 = distances_df.index[0] * distances[-1]
  c2 = distances_df.index[-1] * distances[0]
  c = c1 - c2

  distance_of_points_from_line = []
  for k in range(len(distances_df)):
    distance_of_points_from_line.append(
        calc_distance(distances_df.index[k], distances[k], a, b, c))
    
  dist = pd.Series(distance_of_points_from_line)
  index_max = dist.idxmax()
  return distances[index_max]


In [ ]:
db1 = DBSCAN(eps = find_eps(), min_samples = mins).fit(dummy)

dummy["assignments"] = db1.labels_
color = dummy["assignments"].astype(str)
n_noise_ = list(db1.labels_).count(-1)
print("Estimated number of noise points: %d" % n_noise_)



Estimated number of noise points: 343


In [ ]:
px.scatter_3d(data, x='Hour', y='Temperature (C)', z='Relative Humidity (%)', color = color).update_traces(marker={'size': 2})


In [ ]:
df1 = data.copy()
anomaly = dummy[dummy.assignments == -1]
anomalies_index = list(anomaly.index)
temp_list = []
time_list = []
humidity_list = []

for index in anomalies_index:
  temp = df1.loc[index]['Temperature (C)']
  temp_list.append(temp)
  time = df1.loc[index]['time']
  time_list.append(time)
  humidity = df1.loc[index]['Relative Humidity (%)']
  humidity_list.append(humidity)
dic = {
    'Time': time_list,
    'Temperature': temp_list,
    'Humidity': humidity_list,
}

anomaly_df = pd.DataFrame(dic)
anomaly_df

,Time,Temperature,Humidity
0,2016-10-24 10:00:00,27.87,65.62
1,2016-10-28 18:00:00,23.36,90.33
2,2016-10-28 19:00:00,22.97,91.95
3,2016-10-28 20:00:00,22.66,90.46
4,2016-08-11 02:00:00,20.50,71.48
...,...,...,...
338,2019-10-02 03:00:00,24.64,32.86
339,2019-10-02 04:00:00,26.75,28.42
340,2019-02-22 20:00:00,26.62,55.57
341,2019-02-22 21:00:00,26.18,54.89


In [ ]:
from sklearn import metrics
metrics.silhouette_score(dummy, dummy['assignments'])

0.31778332930705394